User imports

In [391]:
import requests
from bs4 import BeautifulSoup
import csv, sqlite3
import pandas as pd
from scipy import spatial

Scrapping the website

In [392]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [393]:
movie_names = []
movie_links = []
for values in range(5):
    URL = "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page="+str(values)
    user_agent = {'User-agent': 'Mozilla/5.0'}

    response  = requests.get(URL, headers = user_agent)

    soup = BeautifulSoup(response.text, 'html.parser') # Beautiful soup used to display the scrapped internet data

    data = soup.find_all("td", class_="clamp-summary-wrap") # finding the required data in the scrapped data

    for movie in data:
        name = movie.find('h3').text
        movie_names.append(name)
    for movie in data:
        tag = movie.find('a', class_='title')
        link = tag.get('href', None)
        movie_links.append(link)
print(movie_names)
print(movie_links)

['Citizen Kane', 'The Godfather', 'Rear Window', 'Casablanca', 'Boyhood', 'Three Colors: Red', 'Vertigo', 'Notorious', "Singin' in the Rain", 'City Lights', 'Moonlight', 'Intolerance', 'Pinocchio', 'Touch of Evil', 'The Lady Vanishes', 'The Treasure of the Sierra Madre', "Pan's Labyrinth", 'Some Like It Hot', 'North by Northwest', 'Hoop Dreams', 'Rashomon', 'All About Eve', 'Jules and Jim', 'The Wild Bunch', 'My Left Foot', 'The Third Man', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Quo Vadis, Aida?', 'Gone with the Wind', '4 Months, 3 Weeks and 2 Days', 'Psycho', 'Battleship Potemkin', 'A Streetcar Named Desire', 'American Graffiti', 'Dumbo', 'Roma', 'Ran', 'The Shop Around the Corner', '12 Angry Men', 'Manchester by the Sea', "Rosemary's Baby", 'The Maltese Falcon', '12 Years a Slave', 'Killer of Sheep', 'Nashville', 'Ratatouille', 'Parasite', 'Rocks', "Don't Look Now", 'The Grapes of Wrath', 'Children of Paradise (1945)', 'Mean Streets', 'The Lady Eve',

In [394]:
for i in range(len(movie_links)):
    movie_links[i] = 'https://www.metacritic.com'+ movie_links[i]
print(movie_links)

['https://www.metacritic.com/movie/citizen-kane', 'https://www.metacritic.com/movie/the-godfather', 'https://www.metacritic.com/movie/rear-window', 'https://www.metacritic.com/movie/casablanca', 'https://www.metacritic.com/movie/boyhood', 'https://www.metacritic.com/movie/three-colors-red', 'https://www.metacritic.com/movie/vertigo', 'https://www.metacritic.com/movie/notorious-1946', 'https://www.metacritic.com/movie/singin-in-the-rain', 'https://www.metacritic.com/movie/city-lights', 'https://www.metacritic.com/movie/moonlight-2016', 'https://www.metacritic.com/movie/intolerance', 'https://www.metacritic.com/movie/pinocchio-1940', 'https://www.metacritic.com/movie/touch-of-evil', 'https://www.metacritic.com/movie/the-lady-vanishes-1938', 'https://www.metacritic.com/movie/the-treasure-of-the-sierra-madre', 'https://www.metacritic.com/movie/pans-labyrinth', 'https://www.metacritic.com/movie/some-like-it-hot', 'https://www.metacritic.com/movie/north-by-northwest', 'https://www.metacritic

Forming the URL's

In [395]:
detail_links = []
for movie_link in movie_links:
    page = requests.get(movie_link, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    div = soup('div', class_= 'see_all pad_btm1')
    for line in div:
        hyperlink = line.find('a')
        url = hyperlink.get('href', None)
        detail_links.append(url)
print(detail_links)
for i in range(len(detail_links)):
    detail_links[i] = 'https://www.metacritic.com'+ detail_links[i]
print(detail_links)

['/movie/citizen-kane-1941/details', '/movie/the-godfather/details', '/movie/rear-window/details', '/movie/casablanca/details', '/movie/boyhood/details', '/movie/three-colors-red/details', '/movie/vertigo-1958/details', '/movie/notorious-1946/details', '/movie/singin-in-the-rain/details', '/movie/city-lights/details', '/movie/moonlight-2016/details', '/movie/intolerance/details', '/movie/pinocchio-1940/details', '/movie/touch-of-evil/details', '/movie/the-lady-vanishes-1938/details', '/movie/the-treasure-of-the-sierra-madre/details', '/movie/pans-labyrinth/details', '/movie/some-like-it-hot/details', '/movie/north-by-northwest/details', '/movie/hoop-dreams/details', '/movie/rashomon/details', '/movie/all-about-eve/details', '/movie/jules-and-jim/details', '/movie/the-wild-bunch/details', '/movie/my-left-foot/details', '/movie/the-third-man/details', '/movie/dr-strangelove-or-how-i-learned-to-stop-worrying-and-love-the-bomb/details', '/movie/quo-vadis-aida/details', '/movie/gone-with-th

Visualize Scrapped Data

In [396]:
movie_director = dict()
for i in range(len(detail_links)):
    url = detail_links[i]
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    director_table = soup.find('table', class_ = 'credits', summary = movie_names[i] + "'s Director Credits")
    k = str(director_table)
    if (k != 'None'):
        cast_table = soup.find('table', class_ = 'credits', summary = movie_names[i] + "'s Cast Credits")
        j= str(cast_table)
        director_line = director_table.find_all('td', class_ = 'person')
        directors = list()
        for line in director_line:
            hyperlink = line.find('a')
            director = hyperlink.get_text()
            directors.append(director.strip())
        if(j != 'None'):
            p_cast_table = soup.find('table', class_ = 'credits', summary = movie_names[i] + "'s Principal Cast Credits")
            cast_line = cast_table.find_all('td', class_ = 'person')
            casts = list()
            for line in cast_line:
                hyperlink = line.find('a')
                cast = hyperlink.get_text()
                casts.append(cast.strip())
            director_cast = dict()
            director_cast[tuple(directors)] = casts
            movie_director[movie_names[i]] = director_cast
        else:
            director_castc = dict()
            castsc = list()
            castsc.append('null')
            director_castc[tuple(directors)] = castsc
            movie_director[movie_names[i]] = director_castc
    else:
        director_castd = dict()
        directorsd = list()
        directorsd.append('null')
        castsd = list()
        castsd.append('null')
        director_castd[tuple(directorsd)] = castsd
        movie_director[movie_names[i]] = director_castd
print(movie_director)

{'Citizen Kane': {('Orson Welles',): ['Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}, 'The Godfather': {('Francis Ford Coppola',): ['Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}, 'Rear Window': {('Alfred Hitchcock',): ['Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper']}, 'Casablanca': {('Michael Curtiz',): ['Claude Rains', 'Conrad Veidt', 'Curt Bois', 'Dooley Wilson', 'Enrique Acosta', 'John Qualen', 'Joy Page', 'Leonid Kinskey', 'Madeleine Lebeau', 'Paul Henreid', 'Peter Lorre', 'S.Z. Sakall', 'Sydney Greenstreet']}, 'Boyhood': {('Richard Linklater',): ['Alyssa Petersen', 'Andrea Chen', 

In [397]:

all_movies = []
all_movies.append(movie_director)
print(all_movies)
print(type(all_movies))


[{'Citizen Kane': {('Orson Welles',): ['Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}, 'The Godfather': {('Francis Ford Coppola',): ['Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}, 'Rear Window': {('Alfred Hitchcock',): ['Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper']}, 'Casablanca': {('Michael Curtiz',): ['Claude Rains', 'Conrad Veidt', 'Curt Bois', 'Dooley Wilson', 'Enrique Acosta', 'John Qualen', 'Joy Page', 'Leonid Kinskey', 'Madeleine Lebeau', 'Paul Henreid', 'Peter Lorre', 'S.Z. Sakall', 'Sydney Greenstreet']}, 'Boyhood': {('Richard Linklater',): ['Alyssa Petersen', 'Andrea Chen',

Ask the user for input to query the data and give necessary output

In [398]:
try:
    inp_movie = input('Which movie do you want to check? >')
    inp = input('What information about this movie do you want to check? (Choose director or cast) >')

except ValueError:
    print("Please enter valid input")

for movie_dic in all_movies:
    for movie_key, movie_value in movie_dic.items():
        if inp_movie == movie_key:
            if inp == 'director':
                for sub_key, sub_value in movie_value.items():
                    dir = ''
                    for i in sub_key: #remember sub_key is a tuple
                        dir = dir + i + ', '
                    print('The director of the movie', inp_movie, 'is', dir)
            elif inp =='cast':
                for sub_key, sub_value in movie_value.items():
                    cas = ''
                    for i in sub_value: #remember sub_value is a list
                        cas = cas + i + ', '
                    print('The cast of the movie', inp_movie, 'includes', cas)

In [399]:

p = []
for movie_dic in all_movies:
    for movie_key, movie_value in movie_dic.items():
        p.append(movie_key)
        print(movie_key)
        print(movie_value)

Citizen Kane
{('Orson Welles',): ['Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}
The Godfather
{('Francis Ford Coppola',): ['Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}
Rear Window
{('Alfred Hitchcock',): ['Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper']}
Casablanca
{('Michael Curtiz',): ['Claude Rains', 'Conrad Veidt', 'Curt Bois', 'Dooley Wilson', 'Enrique Acosta', 'John Qualen', 'Joy Page', 'Leonid Kinskey', 'Madeleine Lebeau', 'Paul Henreid', 'Peter Lorre', 'S.Z. Sakall', 'Sydney Greenstreet']}
Boyhood
{('Richard Linklater',): ['Alyssa Petersen', 'Andrea Chen', 'Andrew Villarreal',

In [400]:
all_directors = tuple()
k=0
while (k < len(movie_director)):
    for i in all_movies:
        sub = list(movie_director.values())[k] #so this extracts the sub dictionary
        sub_key = list(sub.keys())
        for tpl in sub_key:
            all_directors += tpl
    k = k+1

In [401]:
print(all_directors)


('Orson Welles', 'Francis Ford Coppola', 'Alfred Hitchcock', 'Michael Curtiz', 'Richard Linklater', 'Krzysztof Kieslowski', 'Alfred Hitchcock', 'null', 'Gene Kelly', 'Stanley Donen', 'null', 'Barry Jenkins', 'D.W. Griffith', 'Ben Sharpsteen', 'Bill Roberts', 'Hamilton Luske', 'Jack Kinney', 'Norman Ferguson', 'T. Hee', 'Wilfred Jackson', 'Orson Welles', 'null', 'John Huston', 'Guillermo del Toro', 'Billy Wilder', 'Alfred Hitchcock', 'null', 'Akira Kurosawa', 'Joseph L. Mankiewicz', 'François Truffaut', 'Sam Peckinpah', 'Jim Sheridan', 'Carol Reed', 'Stanley Kubrick', 'Jasmila Zbanic', 'George Cukor', 'Sam Wood', 'Victor Fleming', 'null', 'Alfred Hitchcock', 'Sergei M. Eisenstein', 'Elia Kazan', 'George Lucas', 'Ben Sharpsteen', 'Bill Roberts', 'Jack Kinney', 'John Elliotte', 'Norman Ferguson', 'Samuel Armstrong', 'Wilfred Jackson', 'Alfonso Cuarón', 'Akira Kurosawa', 'Ernst Lubitsch', 'Sidney Lumet', 'Kenneth Lonergan', 'Roman Polanski', 'John Huston', 'Steve McQueen', 'Charles Burnett

Finding the Top director who directed most movies in top 500 movies

In [402]:

all_director_lst = list(all_directors)
dictor_count = dict()
for i in all_director_lst:
    dictor_count[i] = dictor_count.get(i, 0) + 1
print(dictor_count)

dict_values = dictor_count.keys()

print(dict_values)
max_values = max(dict_values)

pop_keys = []
for key, value in dictor_count.items():
    if key == "null":
        pop_keys.append(key)

for key in pop_keys:
    dictor_count.pop(key)

max_cast, max_count = max((v, k) for k, v in dictor_count.items())
print(max_cast, max_count)


{'Orson Welles': 4, 'Francis Ford Coppola': 5, 'Alfred Hitchcock': 8, 'Michael Curtiz': 4, 'Richard Linklater': 3, 'Krzysztof Kieslowski': 2, 'null': 72, 'Gene Kelly': 1, 'Stanley Donen': 1, 'Barry Jenkins': 2, 'D.W. Griffith': 1, 'Ben Sharpsteen': 3, 'Bill Roberts': 4, 'Hamilton Luske': 2, 'Jack Kinney': 2, 'Norman Ferguson': 3, 'T. Hee': 2, 'Wilfred Jackson': 3, 'John Huston': 5, 'Guillermo del Toro': 2, 'Billy Wilder': 3, 'Akira Kurosawa': 2, 'Joseph L. Mankiewicz': 1, 'François Truffaut': 2, 'Sam Peckinpah': 1, 'Jim Sheridan': 1, 'Carol Reed': 2, 'Stanley Kubrick': 4, 'Jasmila Zbanic': 1, 'George Cukor': 4, 'Sam Wood': 1, 'Victor Fleming': 1, 'Sergei M. Eisenstein': 1, 'Elia Kazan': 2, 'George Lucas': 2, 'John Elliotte': 1, 'Samuel Armstrong': 3, 'Alfonso Cuarón': 3, 'Ernst Lubitsch': 1, 'Sidney Lumet': 2, 'Kenneth Lonergan': 1, 'Roman Polanski': 2, 'Steve McQueen': 4, 'Charles Burnett': 1, 'Robert Altman': 4, 'Brad Bird': 1, 'Joon-ho Bong': 1, 'Nicolas Roeg': 1, 'John Ford': 4, 'M

Director **Alfred Hitchcock** was the director for **8** movies in the top 500 movies. We can say that this director was most successful compared to the other directors

Find the actor who acted in most movies in the top 500 movies.

In [403]:

all_casts = list()
m=0
while (m < len(movie_director)):
    for i in all_movies:
        sub = list(movie_director.values())[m] #so this extracts the sub dictionary
        sub_key = list(sub.values())
        for tpl in sub_key:
            all_casts += tpl
    m = m+1

In [404]:
all_cast_lst = list(all_casts)
cast_count = dict()
for i in all_cast_lst:
    cast_count[i] = cast_count.get(i, 0) + 1
print(cast_count)
pop_keys = []
for key, value in cast_count.items():
    if key == "null":
        pop_keys.append(key)

for key in pop_keys:
    cast_count.pop(key)
max_cast, max_count = max((v, k) for k, v in cast_count.items())
print(max_cast, max_count)

{'Agnes Moorehead': 1, 'Dorothy Comingore': 1, 'Erskine Sanford': 1, 'Everett Sloane': 1, 'Fortunio Bonanova': 1, 'George Coulouris': 1, 'Georgia Backus': 2, 'Gus Schilling': 1, 'Harry Shannon': 2, 'Paul Stewart': 1, 'Philip Van Zandt': 1, 'Ray Collins': 1, 'Ruth Warrick': 1, 'William Alland': 1, 'Abe Vigoda': 1, 'Al Lettieri': 1, 'Diane Keaton': 1, 'Gianni Russo': 1, 'James Caan': 2, 'John Cazale': 2, 'John Marley': 1, 'Richard Conte': 1, 'Richard S. Castellano': 1, 'Robert Duvall': 4, 'Rudy Bond': 3, 'Sterling Hayden': 2, 'Talia Shire': 2, 'Havis Davenport': 1, 'Irene Winston': 1, 'Jerry Antes': 1, 'Jesslyn Fax': 1, 'Rand Harper': 1, 'Claude Rains': 1, 'Conrad Veidt': 1, 'Curt Bois': 1, 'Dooley Wilson': 1, 'Enrique Acosta': 1, 'John Qualen': 2, 'Joy Page': 1, 'Leonid Kinskey': 1, 'Madeleine Lebeau': 1, 'Paul Henreid': 1, 'Peter Lorre': 1, 'S.Z. Sakall': 2, 'Sydney Greenstreet': 1, 'Alyssa Petersen': 1, 'Andrea Chen': 1, 'Andrew Villarreal': 1, 'Angela Rawna': 1, 'Bonnie Cross': 1, 'B

Actor/Actress **John Ratzenberger** was casted in **6** movies in the top 500 movies. We can say that this Actor/Actress is in his prime at that point.

In [405]:

#s = ['I', 'want', 4, 'apples', 'and', 18, 'bananas']

# using list comprehension
#listToStr = ' '.join(map(str, s))

#print(listToStr)
all_casts = list()
m=0
castlist =[]
while (m < len(movie_director)):
    for i in all_movies:
        sub = list(movie_director.values())[m] #so this extracts the sub dictionary
        sub_key = list(sub.values())
        for tpl in sub_key:
            all_casts = tpl
            #print(all_casts)
            listToStr = ':'.join(map(str, all_casts))
            castlist.append(listToStr)
            #print(listToStr)

    m = m+1
#print(all_casts)
#print(type(all_casts))
#for i in range(len(movie_director)):
#    listToStr = ' '.join(map(str, all_casts))
#print(listToStr)
print(castlist)
print(type(castlist))

['Agnes Moorehead:Dorothy Comingore:Erskine Sanford:Everett Sloane:Fortunio Bonanova:George Coulouris:Georgia Backus:Gus Schilling:Harry Shannon:Paul Stewart:Philip Van Zandt:Ray Collins:Ruth Warrick:William Alland', 'Abe Vigoda:Al Lettieri:Diane Keaton:Gianni Russo:James Caan:John Cazale:John Marley:Richard Conte:Richard S. Castellano:Robert Duvall:Rudy Bond:Sterling Hayden:Talia Shire', 'Havis Davenport:Irene Winston:Jerry Antes:Jesslyn Fax:Rand Harper', 'Claude Rains:Conrad Veidt:Curt Bois:Dooley Wilson:Enrique Acosta:John Qualen:Joy Page:Leonid Kinskey:Madeleine Lebeau:Paul Henreid:Peter Lorre:S.Z. Sakall:Sydney Greenstreet', 'Alyssa Petersen:Andrea Chen:Andrew Villarreal:Angela Rawna:Bonnie Cross:Brad Hawkins:Cassidy Johnson:David Blackwell:Deanna Brochin:Derek Chase Hickey:Elijah Smith:Ethan Hawke:Evie Thompson:Greg Baglia:Jamie Howard:Jonathan Bell:Jordan Howard:Libby Villari:Lorelei Linklater:Marco Perella:Mark Finn:Megan Devine:Mona Lee Fultz:Natalie Wilemon:Nick Krause:Patric

In [406]:
all_directors = tuple()
k=0
dirlist =[]
while (k < len(movie_director)):
    for i in all_movies:
        sub = list(movie_director.values())[k] #so this extracts the sub dictionary
        sub_key = list(sub.keys())
        for tpl in sub_key:
            all_directors += tpl
            listToStr = ':'.join(map(str, sub_key))
            listToStr = listToStr.replace(',', '')
            listToStr = listToStr.replace('(', '')
            listToStr = listToStr.replace(')', '')
            listToStr = listToStr.replace("'", "")
            dirlist.append(listToStr)
            #print(listToStr)
    k = k+1


print(dirlist)
print(type(dirlist))


['Orson Welles', 'Francis Ford Coppola', 'Alfred Hitchcock', 'Michael Curtiz', 'Richard Linklater', 'Krzysztof Kieslowski', 'Alfred Hitchcock', 'null', 'Gene Kelly Stanley Donen', 'null', 'Barry Jenkins', 'D.W. Griffith', 'Ben Sharpsteen Bill Roberts Hamilton Luske Jack Kinney Norman Ferguson T. Hee Wilfred Jackson', 'Orson Welles', 'null', 'John Huston', 'Guillermo del Toro', 'Billy Wilder', 'Alfred Hitchcock', 'null', 'Akira Kurosawa', 'Joseph L. Mankiewicz', 'François Truffaut', 'Sam Peckinpah', 'Jim Sheridan', 'Carol Reed', 'Stanley Kubrick', 'Jasmila Zbanic', 'George Cukor Sam Wood Victor Fleming', 'null', 'Alfred Hitchcock', 'Sergei M. Eisenstein', 'Elia Kazan', 'George Lucas', 'Ben Sharpsteen Bill Roberts Jack Kinney John Elliotte Norman Ferguson Samuel Armstrong Wilfred Jackson', 'Alfonso Cuarón', 'Akira Kurosawa', 'Ernst Lubitsch', 'Sidney Lumet', 'Kenneth Lonergan', 'Roman Polanski', 'John Huston', 'Steve McQueen', 'Charles Burnett', 'Robert Altman', 'Brad Bird', 'Joon-ho Bon

Placing the data into dataframe

In [407]:
import pandas as pd
data = {'Film': p, 'Director': dirlist, 'Cast': castlist}
new = pd.DataFrame.from_dict(data)

new

,Film,Director,Cast
0,Citizen Kane,Orson Welles,Agnes Moorehead:Dorothy Comingore:Erskine Sanf...
1,The Godfather,Francis Ford Coppola,Abe Vigoda:Al Lettieri:Diane Keaton:Gianni Rus...
2,Rear Window,Alfred Hitchcock,Havis Davenport:Irene Winston:Jerry Antes:Jess...
3,Casablanca,Michael Curtiz,Claude Rains:Conrad Veidt:Curt Bois:Dooley Wil...
4,Boyhood,Richard Linklater,Alyssa Petersen:Andrea Chen:Andrew Villarreal:...
...,...,...,...
492,Room,Lenny Abrahamson,Amanda Brugel:Brad Wietersen:Brie Larson:Cas A...
493,The Tillman Story,Amir Bar-Lev,Pat Tillman:Richard Tillman
494,The Wind Will Carry Us,null,null
495,The Remains of the Day,James Ivory,Abigail Hopkins:Ben Chaplin:Caroline Hunt:Chri...


In [408]:
print(new)


                         Film              Director  \
0                Citizen Kane          Orson Welles   
1               The Godfather  Francis Ford Coppola   
2                 Rear Window      Alfred Hitchcock   
3                  Casablanca        Michael Curtiz   
4                     Boyhood     Richard Linklater   
..                        ...                   ...   
492                      Room      Lenny Abrahamson   
493         The Tillman Story          Amir Bar-Lev   
494    The Wind Will Carry Us                  null   
495    The Remains of the Day           James Ivory   
496  sex, lies, and videotape     Steven Soderbergh   

                                                  Cast  
0    Agnes Moorehead:Dorothy Comingore:Erskine Sanf...  
1    Abe Vigoda:Al Lettieri:Diane Keaton:Gianni Rus...  
2    Havis Davenport:Irene Winston:Jerry Antes:Jess...  
3    Claude Rains:Conrad Veidt:Curt Bois:Dooley Wil...  
4    Alyssa Petersen:Andrea Chen:Andrew Villarreal:...

Finding the cosine similarity

In [409]:
group_data = new
group_data = group_data.drop("Film", axis=1)
group_data = group_data.groupby("Director")
test = group_data.sum()

director_cast_test = test.to_dict()
director_cast_dictionary = director_cast_test.values()
for data in director_cast_dictionary:
    dict_dir_cast = data


dict_values_list = list(dict_dir_cast.values())
dict_key_list = list(dict_dir_cast.keys())


dictionaries = []
for values in dict_values_list:
    cast_split = values.split(":")
    cast_count = dict()
    for i in cast_split:
        cast_count[i] = cast_count.get(i, 0) + 1
    dictionaries.append(cast_count)
dictionary_director_cast_zip = dict(zip(dict_key_list, dictionaries))
print(dictionary_director_cast_zip)
result = []

for key,values in dictionary_director_cast_zip.items():
    value1 = list(values.values())
    for keys,value in dictionary_director_cast_zip.items():
        value2 = list(value.values())
        if value1 != value2 and len(value1) == len(value2):
            result = 1 - spatial.distance.cosine(value1, value2)
            print("The cosine similarity of",value1,"and",value2,"is:",result)


{'Abdel Kechiche': {'Ad': 1, 'Alma Jodorowsky': 1, 'Anne Loiret': 1, 'Aurelie Lemanceau': 1, 'Aurélien Recoing': 1, 'Baya Rehaz': 1, 'Benjamin Siksou': 1, 'Beno': 1, 'Catherine Salée': 1, 'Fanny Maurin': 1, 'Jeremie Laheurte': 1, 'Karim Saidi': 1, 'Léa Seydoux': 1, 'Maelys Cabezon': 1, 'Mona Walravens': 1, 'Salim Kechiouche': 1, 'Samir Bella': 1, 'Sandor Funtek': 1}, 'Abderrahmane Sissako': {'Abel Jafri': 1, 'Adel Mahmoud Cherif': 1, 'Cheik A.G. Emakni': 1, 'Fatoumata Diawara': 1, 'Hichem Yacoubi': 1, 'Ibrahim Ahmed': 1, 'Kettly Noël': 1, 'Layla Walet Mohamed': 1, 'Mamby Kamissoko': 1, 'Mehdi A.G. Mohamed': 1, 'Pino Desperado': 1, 'Salem Dendou': 1, 'Toulou Kiki': 1, 'Weli Cleib': 1, 'Yoro Diakité': 1, 'Zikra Oualet Moussa': 1}, 'Abraham Polonsky': {'Georgia Backus': 1, 'Jessie Arnold': 1, 'Margaret Bert': 1, 'Murray Alper': 1, 'Sam Ash': 1}, 'Adam Schlesinger Linda Saffire': {'Wendy Whelan': 1}, 'Ahmir-Khalib Thompson': {'Gladys Knight & The Pips': 1, 'Mavis Staples': 1, 'Sly and the 

In [410]:
top_5_actors = ['Agnes Moorehead', 'Conrad Veidt', 'Andrea Chen', 'John Ratzenberger', 'Earl T. Taylor']

actors_group = new

casts_in_dict = list(actors_group['Cast'])

top_5_co_actors = []

for values in casts_in_dict:
    casts_in_dict_split = values.split(":")
    for value in top_5_actors:
        if value in casts_in_dict_split:
            top_5_co_actors.append(casts_in_dict_split)

top_5_final = []
for value in top_5_actors:
    top_test = []
    for values in top_5_co_actors:
        if value in values:
            top_test = top_test + values
    top_5_final.append(top_test)

top_all_dict = []

for keys in top_5_final:
    top_actor_count = dict()
    for key in keys:
        top_actor_count[key] = top_actor_count.get(key, 0) + 1
    top_all_dict.append(top_actor_count)

top_final_dist = dict(zip(top_5_actors, top_all_dict))

print(top_final_dist)
size_list = []
for key,values in top_final_dist.items():
    size_list.append(len(list(values.values())))
size = max(size_list)

for key,values in top_final_dist.items():
    value1 = list(values.values())
    for i in range(size):
        if len(value1) < size:
            value1.append(0)
    for keys,value in top_final_dist.items():
        value2 = list(value.values())
        for i in range(size):
            if len(value2) < size:
                value2.append(0)
        if value1 != value2 and len(value1) == len(value2):
            result = 1 - spatial.distance.cosine(value1, value2)
            print("The cosine similarity of",value1,"and",value2,"is:",result)


{'Agnes Moorehead': {'Agnes Moorehead': 1, 'Dorothy Comingore': 1, 'Erskine Sanford': 1, 'Everett Sloane': 1, 'Fortunio Bonanova': 1, 'George Coulouris': 1, 'Georgia Backus': 1, 'Gus Schilling': 1, 'Harry Shannon': 1, 'Paul Stewart': 1, 'Philip Van Zandt': 1, 'Ray Collins': 1, 'Ruth Warrick': 1, 'William Alland': 1}, 'Conrad Veidt': {'Claude Rains': 1, 'Conrad Veidt': 1, 'Curt Bois': 1, 'Dooley Wilson': 1, 'Enrique Acosta': 1, 'John Qualen': 1, 'Joy Page': 1, 'Leonid Kinskey': 1, 'Madeleine Lebeau': 1, 'Paul Henreid': 1, 'Peter Lorre': 1, 'S.Z. Sakall': 1, 'Sydney Greenstreet': 1}, 'Andrea Chen': {'Alyssa Petersen': 1, 'Andrea Chen': 1, 'Andrew Villarreal': 1, 'Angela Rawna': 1, 'Bonnie Cross': 1, 'Brad Hawkins': 1, 'Cassidy Johnson': 1, 'David Blackwell': 1, 'Deanna Brochin': 1, 'Derek Chase Hickey': 1, 'Elijah Smith': 1, 'Ethan Hawke': 1, 'Evie Thompson': 1, 'Greg Baglia': 1, 'Jamie Howard': 1, 'Jonathan Bell': 1, 'Jordan Howard': 1, 'Libby Villari': 1, 'Lorelei Linklater': 1, 'Marco

Placing data into CSV file

In [411]:
new.to_csv('movies.csv')

Placing data into Database

In [361]:
con = sqlite3.connect("MovieInfoDatabase.db")
cur = con.cursor()# use your column names here
movie_table = """ CREATE TABLE IF NOT EXISTS MovieInfoTable (
                                        Movie_Number integer PRIMARY KEY,
                                        Film text,
                                        Director text,
                                        Casts text
                                    ); """
cur.execute(movie_table)

csv_data = pd.read_csv(r"movies.csv")
df = pd.DataFrame(csv_data)
df.rename( columns={'Unnamed: 0':'Index'}, inplace=True)
print(df)

insert = '''INSERT INTO MovieInfoTable (Movie_Number, Film, Director, Casts) VALUES (?,?,?,?);'''

for index, rows_value in df.iterrows():
    rows_value = list(rows_value)
    cur.execute(insert,rows_value)
con.commit()
con.close()

     Index                      Film               Dirctor  \
0        0              Citizen Kane          Orson Welles   
1        1             The Godfather  Francis Ford Coppola   
2        2               Rear Window      Alfred Hitchcock   
3        3                Casablanca        Michael Curtiz   
4        4                   Boyhood     Richard Linklater   
..     ...                       ...                   ...   
492    492                      Room      Lenny Abrahamson   
493    493         The Tillman Story          Amir Bar-Lev   
494    494    The Wind Will Carry Us                   NaN   
495    495    The Remains of the Day           James Ivory   
496    496  sex, lies, and videotape     Steven Soderbergh   

                                                  Cast  
0    Agnes Moorehead:Dorothy Comingore:Erskine Sanf...  
1    Abe Vigoda:Al Lettieri:Diane Keaton:Gianni Rus...  
2    Havis Davenport:Irene Winston:Jerry Antes:Jess...  
3    Claude Rains:Conrad Ve